In [1]:
from scipy.io import wavfile
import os
import numpy as np
import pandas as pd

SETUP

In [2]:
# Variables you may change
numberSpikes = 20 # number of integer multiples of the fundamental frequency to record (including 0 Hz).
FFTSize = 8192 # have to use a multiple of 2 for optimal speed.

#open directory with the samples in it.
os.chdir('./Training_data/')

# next, make a NP array of every wav file in the directory
SampleList = np.array(os.listdir())
SampleList = SampleList[ np.char.endswith(SampleList, '.wav') ] #ignore everything that's not a .wav file.

#make a fresh blank series.
SampleSpectra = pd.DataFrame(columns = np.arange(numberSpikes))

CORE LOOP

In [3]:
for SampleName in SampleList:
    SampleRate, Audiodata = wavfile.read(SampleName)
    # spectrum
    from scipy.fft import fft # fourier transform
    m = len(Audiodata) 
    #AudioFreq = fft(Audiodata,n=FFTSize,axis=0)
    AudioFreq = fft(Audiodata[:,1]*np.hanning(np.shape(Audiodata)[0]),n=FFTSize,axis=0) #with a Hanning window
    AudioFreq = AudioFreq[0:int(np.ceil((np.shape(AudioFreq)[0])/2.0))] #Left half of the spectrum
    MagFreq = np.abs(AudioFreq) # Magnitude
    MagFreq = MagFreq / float(m)
    # power spectrum
    MagFreq = MagFreq**2
    if m % 2 > 0: # ffte odd 
        MagFreq[1:len(MagFreq)] = MagFreq[1:len(MagFreq)] * 2
    else:# fft even
        MagFreq[1:len(MagFreq) -1] = MagFreq[1:len(MagFreq) - 1] * 2 
    # HPS (Harmonic Product Spectrum) Algorithm to determine fundamental frequency
    HPSSize = int(np.ceil(len(MagFreq))/3.0 + 1)
    MagFreq2 = MagFreq.copy().reshape(int(np.ceil(len(MagFreq))/2.0),2).mean(1)[0:HPSSize]
    MagFreq3 = np.concatenate([MagFreq.copy(),np.zeros(3 - int(len(MagFreq)) % 3)]).reshape(int(np.ceil(len(MagFreq))/3.0 + 1),3).mean(1)[0:HPSSize]
    HPS = MagFreq[0:HPSSize]*MagFreq2*MagFreq3
    freqAxis = np.arange(0,int(np.ceil((m+1)/2.0)), int(np.ceil((m+1)/2.0))*2/FFTSize) * (SampleRate / m);
    freqAxisDownscaled = np.arange(0,int(np.ceil((m+1)/2.0)), int(np.ceil((m+1)/2.0))*6/FFTSize) * (SampleRate / m);
    FFLocHPS = np.argmax(HPS[freqAxisDownscaled >= 50]) + int(len(HPS[freqAxisDownscaled < 50])) # don't look below 50 Hz for it, due to noise.
    FFLocation = FFLocHPS*3 - 1 # FFLocHPS uses an index that is scaled down by 1/3. Have to reverse that.
    # Have to ensure the fundamental frequency is accurate. Take loudest spike, get its approximate ratio, round to nearest int.
    MaxLoc = np.argmax(MagFreq[freqAxis >= 50]) + int(len(MagFreq[freqAxis < 50])) # don't look below 50 Hz for it, due to noise.
    MaxRatio = np.max([np.round(freqAxis[MaxLoc]/freqAxis[FFLocation]), 1]) 
    FFreq = freqAxis[MaxLoc]/MaxRatio
    MagFreqLog = 10*np.log10(MagFreq)
    FFreqLoc = np.argmin(np.abs(freqAxis - FFreq))
    # If the frequency on either side of the "spike" is actually louder, make that the new spike.
    FFreqLoc = FFreqLoc - ((MagFreqLog <= np.roll(MagFreqLog,1)) & (np.roll(MagFreqLog,-1) <= np.roll(MagFreqLog,1))) #if the left entry is >= the entry and the right entry, then make the left entry the new spike location
    FFreqLoc = FFreqLoc + ((MagFreqLog <= np.roll(MagFreqLog,-1)) & (np.roll(MagFreqLog,1) <= np.roll(MagFreqLog,-1)))  #if the right entry is >= the entry and the left entry, then make the right entry the new spike location
    Multiples = [FFreq*i for i in np.arange(0,numberSpikes)]
    MultiplesLoc = [np.argmin(np.abs(freqAxis - i)) for i in Multiples]
    # Next, make an integer ratio-indexed list of the amplitudes!
    MultiplesMag = MagFreq[MultiplesLoc]
    MultiplesdB = MagFreqLog[MultiplesLoc]
    # Write these magnitudes/dBs to the Pandas array! (Index = SampleName)
    tempIndex = pd.MultiIndex.from_product([[SampleName],['Pwr','dB']])
    SampleRow = pd.DataFrame(np.vstack([MultiplesMag, MultiplesdB]), index = tempIndex, columns = np.arange(0,numberSpikes))
    SampleSpectra = pd.concat([SampleSpectra, SampleRow])

CSV FILE OUTPUT

In [4]:
SampleSpectra.to_csv('000SampleSpectra.csv')

print('The first few samples\' spectra for your viewing pleasure:')
print(SampleSpectra.head())

The first few samples' spectra for your viewing pleasure:
                                         0            1             2   \
(01 - Circle of Life_0.wav, Pwr)  61.014667  1112.303713  1.220439e-02   
(01 - Circle of Life_0.wav, dB)   17.854342    30.462234 -1.913484e+01   
(01 - Circle of Life_1.wav, Pwr)   0.044072   101.640445  1.006199e+01   
(01 - Circle of Life_1.wav, dB)  -13.558345    20.070666  1.002684e+01   
(01 - Circle of Life_2.wav, Pwr)   0.000002     0.000109  7.362571e-07   

                                         3          4          5          6   \
(01 - Circle of Life_0.wav, Pwr)   0.003119   0.001861   0.001438   0.001008   
(01 - Circle of Life_0.wav, dB)  -25.059827 -27.301782 -28.422371 -29.965447   
(01 - Circle of Life_1.wav, Pwr)  14.333925   3.912804   0.825094   0.333062   
(01 - Circle of Life_1.wav, dB)   11.563651   5.924881  -0.834964  -4.774746   
(01 - Circle of Life_2.wav, Pwr)   0.000005   0.000005   0.000005   0.000005   

                